# Calculate upper-tropospheric stability

UTS defined as difference of potential temperature between the dynamic tropopause (defined as the 2PVU level) and 3 km below the 2PVU level similar to Li et al, JGR, 2014, doi: 10.1002/2013JD020669

In [ ]:
import xarray as xr
import dask
import shutil
from pathlib import Path
#for time and memory impression:
import psutil
import time as tm
import datetime 
import numpy as np

In [ ]:
from dask.distributed import Client
client = Client()
client

In [ ]:
# constants declaration, values taken from ICON source code
Cp = 1004.64 #J kg K-1
Ra = 287.04  #J kg-1 K-1

Load dictionary will all simulations

In [ ]:
import sys
sys.path.append('/pf/b/b380459/nawdex-hackathon/')
import dict_nawdexsims
simdict   = dict_nawdexsims.simdictionary()

In [ ]:
def def_memory(msg=None):
    process = psutil.Process()
    mem = np.round(process.memory_info().rss/(1024*1024))
    return mem

In [ ]:
def load_data(expid):
    
    # load data
    path  = '/scratch/b/b380459/icon_4_hackathon/'
    fname = path+'/'+expid+'/'+expid+'_2016*_fg_DOM01_ML_*.nc'
    ds_fg = ( xr.open_mfdataset(fname, combine='by_coords',parallel=True, 
                                engine='h5netcdf', chunks={'time': 1})
                                [['temp', 'pres', 'z_ifc']].rename({'ncells_2': 'ncells', 'height_3': 'height_ifc'}) )
    fname = path+'/'+expid+'/'+expid+'_2016*_pv_DOM01_ML_*.nc'
    ds_pv = ( xr.open_mfdataset(fname, combine='by_coords', parallel=True, 
                                engine='h5netcdf', chunks={'time': 1})
                                [['pv']] )
    
    # merge datasets
    ds = xr.merge([ds_pv, ds_fg])
    del(ds_pv, ds_fg)
    
    # compute height at full levels
    # !!! ATTENTION !!!
    ds['dz'] = -1*ds['z_ifc'].diff(dim='height_ifc', n=1, label='lower').rename({'height_ifc': 'height'})
    ds['z']  = ( ds['z_ifc'].isel(height_ifc=slice(0,75)).rename({'height_ifc': 'height'}) 
                 - 0.5*ds['dz'].isel(height=slice(0,75)) )
    ds = ds.drop(['z_ifc'])
    
    # change units to PVU
    ds['pv'] = ds['pv'] * 1000000
    ds.pv.attrs['units']='PVU'
  
    return ds

In [ ]:
def load_data_timeloop(expid,tstep):
      
    # load data
    path  = '/scratch/b/b380459/icon_4_hackathon/'
    fname = path+'/'+expid+'/'+expid+'_2016*_fg_DOM01_ML_'+str(tstep).zfill(4)+'.nc'
    #print(fname)
    ds_fg = ( xr.open_mfdataset(fname, combine='by_coords',parallel=True, 
                                engine='h5netcdf', chunks={'ncells_2': 1e6})
                                [['temp', 'pres', 'z_ifc']].rename({'ncells_2': 'ncells', 'height_3': 'height_ifc'}) )
    fname = path+'/'+expid+'/'+expid+'_2016*_pv_DOM01_ML_'+str(tstep).zfill(4)+'.nc'
    #print(fname)
    ds_pv = ( xr.open_mfdataset(fname, combine='by_coords', parallel=True, 
                                engine='h5netcdf', chunks={'ncells': 1e6})
                                [['pv']] )
    
    # merge datasets
    ds = xr.merge([ds_pv, ds_fg])
    del(ds_pv, ds_fg)
    
    # compute height at full levels
    # !!! ATTENTION !!!
    ds['dz'] = -1*ds['z_ifc'].diff(dim='height_ifc', n=1, label='lower').rename({'height_ifc': 'height'})
    ds['z']  = ( ds['z_ifc'].isel(height_ifc=slice(0,75)).rename({'height_ifc': 'height'}) 
                 - 0.5*ds['dz'].isel(height=slice(0,75)) )
    ds = ds.drop(['z_ifc'])
    
    # change units to PVU
    ds['pv'] = ds['pv'] * 1000000
    ds.pv.attrs['units']='PVU'
  
    return ds

In [ ]:
def calc_uts(heightlim_top, heightlim_bot):
    
    #index, height and theta at 2PVU level
    #NOTE: a (subjective) height limit is set, in order to avoid 2PVU features in the troposphere. 
    ds['ind_tropheight'] = (( abs(ds.pv.isel(height=slice(heightlim_top,heightlim_bot)) - 2) ).argmin(dim='height') + heightlim_top )
    ds['ind_tropheight'] = ds['ind_tropheight'].where(ds['ind_tropheight'] != heightlim_top, 0)
    ds['tropheight'] = ds['z'].where(ds.height == ds['ind_tropheight'],-999).max(dim='height')
    ds['theta_trop'] = ds['theta'].where(ds.height == ds['ind_tropheight'],-999).max(dim='height')

    #index, height and theta at the level 3km below 2PVU level
    ds['ind_lowheight'] = abs(ds['z'] - (ds['tropheight']-3000)).argmin(dim='height')
    ds['lowheight'] = ds['z'].where(ds.height == ds['ind_lowheight'],-999).max(dim='height')
    ds['theta_low'] = ds['theta'].where(ds.height == ds['ind_lowheight'],-999).max(dim='height')   

    # upper tropospheric stability [K/km]
    ds['uts'] = (ds['theta_trop'] - ds['theta_low']) / ( (ds['tropheight'] - ds['lowheight']) /1000 )
    ds['uts'] = ds['uts'].where(ds['uts'] < 100, -10)
    
    return ds['uts']

In [ ]:
for sim in list(simdict.keys()):
    mem_before = def_memory()
    time_start = tm.time()
    # zarr store
    zarr_store = '/scratch/b/b380459/icon_4_hackathon/upper_tropo_stability/uts_'+sim+'.zarr'
    # remove any zarr_store with same name that might have been created previously
    shutil.rmtree(zarr_store, ignore_errors=True) 
    
    if sim[0:13] == 'nawdexnwp-2km':
        print('Working on:', sim)
        print('      Starting at: ', datetime.datetime.now().time())
        # create sorted list of available timesteps
        tstep_list = []
        for i in Path('/scratch/b/b380459/icon_4_hackathon/'+sim).rglob(sim+'_2016*_fg_DOM01_ML_*.nc'):
            tstep_list.append(str(i).split('/')[-1].split('_')[5].split('.')[0])
        tstep_list.sort(key=int)
        # loop over all available timesteps, provided the timestep list is not empty
        if tstep_list:
            for tstep in tstep_list:
                ds = load_data_timeloop(sim, tstep)
                ds['theta'] = ds['temp'] * (100000./ds['pres'])**(Ra/Cp)
                ds['UTS'] = calc_uts(15,50)
                ds = ds.drop(['temp','pres','theta','pv','dz','z'])
                ds = ds.drop(['ind_tropheight','tropheight','theta_trop','ind_lowheight','lowheight','theta_low'])
                ds.attrs['units'] = 'Kelvin km-1'
                ds.attrs['description'] = 'upper tropospheric stability'
                ds.attrs['simulation'] = sim
                
                # if this is the first timestep, then create new zarr store, otherwise append to existing zarr store
                if Path(zarr_store).exists():
                    ds[['UTS']].to_zarr(zarr_store, append_dim='time')
                else:
                    ds[['UTS']].to_zarr(zarr_store)
    
    else:
        print('Working on:', sim)
        ds = load_data(sim)
        ds['theta'] = ds['temp'] * (100000./ds['pres'])**(Ra/Cp)
        ds['UTS'] = calc_uts(15,50)
        ds = ds.drop(['temp','pres','theta','pv','dz','z'])
        ds = ds.drop(['ind_tropheight','tropheight','theta_trop','ind_lowheight','lowheight','theta_low'])
        ds.attrs['units'] = 'Kelvin km-1'
        ds.attrs['description'] = 'upper tropospheric stability'
        ds.attrs['simulation'] = sim
        # store to zarr store
        ds[['UTS']].to_zarr(zarr_store)
    
    mem_after = def_memory()
    time_elapsed = (tm.time() - time_start) /60.
    print('      Memory before and after loading data: ', mem_before,'MB --> ', mem_after,'MB') 
    print('      Time elapsed (min): ', time_elapsed)
    

Clean up

In [ ]:
client.close()